In [ ]:
from jdaviz.app import Application

app = Application(configuration='cubeviz')
app

In [ ]:
from astropy.utils.data import download_file

fn = download_file('https://data.sdss.org/sas/dr14/manga/spectro/redux/v2_1_2/7495/stack/manga-7495-12704-LOGCUBE.fits.gz', cache=True)
app.load_data(fn)

In [ ]:
v = list(app.components.get('g-viewer-area').items[0].stacks[0].stacks[1].stacks[0].stacks)[0] 
v

This is a *very* non-trivial way to access the viewers.  And there's still no way to get the data per se.  So the below is now the *desired* workflow instead of a working example.

## Desired workflow for getting the spectrum from cubeviz

In [ ]:
from astropy.io import fits

fobj = fits.open(fn)
app.load_data(fobj, name='my manga cube')

(The above currently does not work, only file paths work, and the dataset name comes from the filename)

Now I select a part of the data in one of the top three panels, which in cubeviz yields a spectrum in the bottom panel. Two ways of then interacting programmatically come to mind:

###  Option 1

 The following should work:

In [ ]:
spec = app.get_spectrum()  # <- this gets the "most recent" plot in the spectrum viewer, as a Spectrum1D
spec.spectral_axis, spec.flux

In [ ]:
from specutils.manipulation import box_smooth

smoothed_spec = box_smooth(spec, 5)

app.add_spectrum(smoothed_spec)

Now the smoothed spectrum should appear in the spectrum view.


### Option 2

An alternative approach could be to make it a bit more viewer-centric:

In [ ]:
spec = app.get_viewer('spectrum').get_spectrum() #<- needs to be a Spectrum1D

smoothed_spec = box_smooth(spec, 5)

app.get_viewer('spectrum').add_layerordataorsomething(smoothed_spec)

I prefer option 1 because it keeps things neat and clean for the science case, but it's not as generalizable.

*After some further discussion:* Option 2 seems better because it fits much better into a more general glue scheme, which is desirable because it gives users a more natural way to learn how to unlock more advanced features of glupyter.

## Region selection

I'm going to continue using an "option 1"-like idiom, but that could be modified to Option 2 if we go that route.

Now I select a region in the *spectrum* that spans some feature I'm interested in - say an emission line.  I should be able to do:

In [ ]:
reg = app.get_spectral_selection()  # <- `reg` is a specutils.SpectralRegion

In [ ]:
from specutils.analysis import line_flux

flux = line_flux(spec, reg) 
flux

Which should yield the flux of the region I just selected. 

## Pass on back to the cube view

Now armed with that region selection, I should be able to use the region I extracted to interact with the image viewer UI.  Something like:

In [ ]:
subcube = app.get_cube().spectral_slab(reg)
momap = subcube.moment1()

app.show_cube(momap)

Which should then update the three cube views to be showing a layer that is the first-moment in the relevant spectral region.

# Notes for UI/UX changes:

* The "Test X" tabs take up valuable real-estate and should be removed or hidden
* There should be "quick links" to the layout elements like app.spec_viewer, app.cube_viewer_1, app.cube_viewer_2, etc.  Should be a very transparent mapping to the glue-standard viewer names, though - e.g. `app.spectrum_viewer` should be syntactic sugar for `app.viewers['spectrum-viewer']`
* Maybe a reasonable "auto-selection" option should get applied.  E.g., when I select in the flux window, it should auto-sum that selection and immediately show it in the spectrum view. Or if there was an auto-selection on for the spectrum view, it might auto-median-combine (although I wouldn't say that's a desirable auto-select).